In [1]:
import pandas as pd
import sys
import tqdm
import os
import itertools
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df = pd.read_excel('2016-2020.xlsx')

In [3]:
# wybranie rejonu, mozliwe rejony to RG RZ i RP
df_sorted_data = df.loc[df['REJON']=='RG'].sort_values(['DATA', 'GODZ', 'MIN'])
df_sorted_data.head(10)

,NR,DATA,GODZ,MIN,SEK,ENG,X,Y,Z,REJON,ODDZIAL,POLE,TYP,UWAGI
0,1,2016.01.01,1,0,24,30,0,0,0,RG,G-2,RU-XVII/2,SL,NaN
3,4,2016.01.01,3,23,47,40,0,0,0,RG,G-3,G-1/7,SL,NaN
4,5,2016.01.01,3,23,48,40,0,0,0,RG,G-3,G-1/7,SL,NaN
5,6,2016.01.01,4,29,34,40,0,0,0,RG,G-7,G-7/5,SL,NaN
6,7,2016.01.01,6,5,13,30,0,0,0,RG,G-2,RU-XVII/2,SL,NaN
7,8,2016.01.01,6,14,40,40,0,0,0,RG,G-2,RU-XVII/2,SL,NaN
8,9,2016.01.01,7,35,38,9300,5706037,5577895,-751,RG,G-7,G-7/5,W,FP/CW
9,10,2016.01.01,8,7,32,40,0,0,0,RG,G-3,G-1/7,SL,NaN
10,11,2016.01.01,8,12,59,40,0,0,0,RG,G-2,RU-XVII/2,SL,NaN
11,12,2016.01.01,8,13,3,40,0,0,0,RG,G-2,RU-XVII/2,SL,NaN


In [4]:
df_sorted_data.ODDZIAL.unique() # jakie oddzialy mozemy wybrac w danym rejonie

array(['G-2', 'G-3', 'G-7', 'G-1', 'TW226', 'G-4', 'G-13', 'G-6', 'G-15',
       'G-12', 'G-11', 'TW227a', 'G-5/1'], dtype=object)

In [5]:
# Wytrenowanie modelu moze odbyc sie dla każdego oddzialu pod warunkiem że ma wystarczającą ilość zdarzen o wysokiej energii, zalozono ponad 100 wydarzeń
oddzial = 'G-1'
df_g_1 = df_sorted_data[df['ODDZIAL'] == oddzial]
df_g_1 = df_g_1.reset_index(drop=True)
big_eng_ind = df_g_1.index[df_g_1['ENG'] >10000].tolist() # indexes of all energies higher than 10k
amount_of_high_energy_incidents = len(big_eng_ind)

c:\python37\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
''' Zakladam 
Wejscie do modelu to 5 zarejestrowanych przed wystapienem wstrzasu o wysokiej energii trzesien oraz ich energie, wraz z przedzialami godzinowymi 

Wyjscie modelu
Klasyfikacja czy w następnym przedziale czasowym nastąpi aktywnośc sejsmiczna 1/0
'''
train_df = pd.DataFrame(columns = [ 
                                   'prev1_eng','prev2_eng','prev3_eng','prev4_eng','prev5_eng',
                                   'prev1_time_0:8','prev1_time_8:16','prev1_time_16:24',
                                   'prev2_time_0:8','prev2_time_8:16','prev2_time_16:24',
                                   'prev3_time_0:8','prev3_time_8:16','prev3_time_16:24',
                                   'prev4_time_0:8','prev4_time_8:16','prev4_time_16:24',
                                   'prev5_time_0:8','prev5_time_8:16','prev5_time_16:24',
                                   'prev1_TYP W','prev1_TYP SL','prev1_TYP O','prev1_TYP T',
                                   'prev2_TYP W','prev2_TYP SL','prev2_TYP O','prev2_TYP T',
                                   'prev3_TYP W','prev3_TYP SL','prev3_TYP O','prev3_TYP T',
                                   'prev4_TYP W','prev4_TYP SL','prev4_TYP O','prev4_TYP T',
                                   'prev5_TYP W','prev5_TYP SL','prev5_TYP O','prev5_TYP T',
                                   'MAJOR_EVENT'
                                   ])
"""
Pętla ma na celu wyszukanie wszystkich zdarzen o energi większej od 1*10^5 J
Nastepnie dodaje każde takie wydarzenie do tablicy wraz z pięcioma poprzednimi
odczytami. W ten sposób tworzone jest wejscie do modelu skladajace sie
z 40 kolumn
"""
for current_index in big_eng_ind:
  
  train_row = []
  energies = []
  time =[]

  for i in range(5):
    temp = df_g_1.loc[current_index - i - 1, :].ENG
    energies.append(temp)

  for j in range(5):
    temp = df_g_1.loc[current_index-j-1,:]
    row_to_append = [0, 0, 0]
    if temp['GODZ'] < 8:
      row_to_append[0] = 1
    elif temp['GODZ'] < 16:
      row_to_append[1] = 1
    elif temp['GODZ'] < 24:
      row_to_append[2] = 1
    time.append(row_to_append)

  type_of_event = []
  for k in range(5):
    row_to_append = [0, 0, 0, 0]
    temp = df_g_1.loc[current_index-k-1,:]
    if temp['TYP'] == 'W':
      row_to_append[0] = 1
    elif temp['TYP'] == 'SL':
      row_to_append[1] = 1
    elif temp['TYP'] == 'O':
      row_to_append[2] = 1
    elif temp['TYP'] == 'T':
      row_to_append[3] = 1
    type_of_event.append(row_to_append)

  for energy in energies:
    train_row.append(energy)

  # time is a nested list, each of nested list stores value if the event occured
  # beetween 0-8 8-16 16-24 and its occurance marks it as 1. got to unwind nested list 
  # to put it inside of row 
  for daytime in time :
    for time_of_the_day in daytime:
      train_row.append(time_of_the_day)

  for prev_event in type_of_event:
    for num_symbolic_event in prev_event:
      train_row.append(num_symbolic_event)

  train_row.append(1)

  train_df = train_df.append(pd.DataFrame(
      [train_row],
      columns = [                    'prev1_eng','prev2_eng','prev3_eng','prev4_eng','prev5_eng',
                                    'prev1_time_0:8','prev1_time_8:16','prev1_time_16:24',
                                    'prev2_time_0:8','prev2_time_8:16','prev2_time_16:24',
                                    'prev3_time_0:8','prev3_time_8:16','prev3_time_16:24',
                                    'prev4_time_0:8','prev4_time_8:16','prev4_time_16:24',
                                    'prev5_time_0:8','prev5_time_8:16','prev5_time_16:24',
                                    'prev1_TYP W','prev1_TYP SL','prev1_TYP O','prev1_TYP T',
                                    'prev2_TYP W','prev2_TYP SL','prev2_TYP O','prev2_TYP T',
                                    'prev3_TYP W','prev3_TYP SL','prev3_TYP O','prev3_TYP T',
                                    'prev4_TYP W','prev4_TYP SL','prev4_TYP O','prev4_TYP T',
                                    'prev5_TYP W','prev5_TYP SL','prev5_TYP O','prev5_TYP T',
                                    'MAJOR_EVENT' ]),
                                    ignore_index = True)




In [ ]:
from random import randint
import numpy as np
"""
Funkcja upewnia sie, że przypadkowe wybrane zdarzenie z zbioru danych nie zawieralo
w poblizu 5 pomiarow odczytu o wartosci wiekszej niz 10^5 J
"""
def closest(lst, K):
      
     lst = np.asarray(lst)
     idx = (np.abs(lst - K)).argmin()
     return lst[idx]

random_value = randint(0,len(df_g_1))

"""
Petla odpowiada za dodanie zdarzen do zbioru trenujacego gdzie nie wystapilo 
wydarzenie o znaczącej sile. Ilosc danych jest znacząco przewazajaca dla nieznaczacych 
wydarzen wobec tego, ilosc dobranych wydarzen nieznaczacych odpowiada ilosci znaczacych
w zbiorze testowym
"""
for iterator in range(amount_of_high_energy_incidents):
  random_value = randint(0,len(df_g_1))
  closest_value = closest(big_eng_ind, random_value)
  if closest_value <= 5:
    pass
  else:
    current_index = random_value
    
    train_row = []
    energies = []
    time =[]

    for i in range(5):
      temp = df_g_1.loc[current_index - i - 1, :].ENG
      energies.append(temp)

    for j in range(5):
      temp = df_g_1.loc[current_index-j-1,:]
      row_to_append = [0, 0, 0]
      if temp['GODZ'] < 8:
        row_to_append[0] = 1
      elif temp['GODZ'] < 16:
        row_to_append[1] = 1
      elif temp['GODZ'] < 24:
        row_to_append[2] = 1
      time.append(row_to_append)

    type_of_event = []
    for k in range(5):
      row_to_append = [0, 0, 0, 0]
      temp = df_g_1.loc[current_index-k-1,:]
      if temp['TYP'] == 'W':
        row_to_append[0] = 1
      elif temp['TYP'] == 'SL':
        row_to_append[1] = 1
      elif temp['TYP'] == 'O':
        row_to_append[2] = 1
      elif temp['TYP'] == 'T':
        row_to_append[3] = 1
      type_of_event.append(row_to_append)

    for energy in energies:
      train_row.append(energy)
    # time is a nested list, each of nested list stores value if the event occured
    # beetween 0-8 8-16 16-24 and its occurance marks it as 1. got to unwind nested list 
    #to put it inside of row 
    for daytime in time :
      for time_of_the_day in daytime:
        train_row.append(time_of_the_day)

    for prev_event in type_of_event:
      for num_symbolic_event in prev_event:
        train_row.append(num_symbolic_event)

    train_row.append(0)

    train_df = train_df.append(pd.DataFrame(
        [train_row],
        columns = [                    'prev1_eng','prev2_eng','prev3_eng','prev4_eng','prev5_eng',
                                      'prev1_time_0:8','prev1_time_8:16','prev1_time_16:24',
                                      'prev2_time_0:8','prev2_time_8:16','prev2_time_16:24',
                                      'prev3_time_0:8','prev3_time_8:16','prev3_time_16:24',
                                      'prev4_time_0:8','prev4_time_8:16','prev4_time_16:24',
                                      'prev5_time_0:8','prev5_time_8:16','prev5_time_16:24',
                                      'prev1_TYP W','prev1_TYP SL','prev1_TYP O','prev1_TYP T',
                                      'prev2_TYP W','prev2_TYP SL','prev2_TYP O','prev2_TYP T',
                                      'prev3_TYP W','prev3_TYP SL','prev3_TYP O','prev3_TYP T',
                                      'prev4_TYP W','prev4_TYP SL','prev4_TYP O','prev4_TYP T',
                                      'prev5_TYP W','prev5_TYP SL','prev5_TYP O','prev5_TYP T',
                                      'MAJOR_EVENT' ]),
                                      ignore_index = True)



 


In [ ]:
train_df.head(10)


In [ ]:
###### preparing data to be innputed into the model
shuffled_train_df = train_df.sample(frac=1).reset_index(drop=True)
shuffled_train_df

In [ ]:

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC



x = shuffled_train_df.drop(columns = ['MAJOR_EVENT'])
y = shuffled_train_df['MAJOR_EVENT']


X_train, X_test, y_train, y_test = train_test_split(x,
                                                    y,
                                                    test_size=.2)
"""
Zdecydowano sie na uzycie 3 modeli SVM o roznych jądrach 
tj. radial basis function, wielomianowego oraz liniowego
"""
svm_rbf = SVC(kernel='rbf', probability=True, random_state=42)
svm_poly = SVC(kernel='poly', probability=True, random_state=42)
svm_lin = SVC(kernel='linear', probability=True, random_state=42)

#svm potrzebuje jako wejscia listy 
x_train_list = X_train.values.tolist()
x_test_list = X_test.values.tolist()
y_train_list = y_train.values.tolist()
y_test_list = y_test.values.tolist()

# skalowanie danych zawierajacych energie wstrzasu
scaler = StandardScaler()
scaler.fit(x_train_list)
x_train_scalled = (scaler.transform(x_train_list))
x_test_scalled = (scaler.transform(x_test_list))

svm_rbf.fit(x_train_scalled, y_train_list)
svm_poly.fit(x_train_scalled, y_train_list)
svm_lin.fit(x_train_scalled, y_train_list)

In [ ]:
# Pomiar poprawnosci przewidywania
y_pred_rbf = svm_rbf.predict(x_test_scalled)
y_pred_poly = svm_poly.predict(x_test_scalled)
y_pred_lin = svm_lin.predict(x_test_scalled)

accuracy_rbf = accuracy_score(y_test_list, y_pred_rbf)
accuracy_poly = accuracy_score(y_test_list, y_pred_poly)
accuracy_lin = accuracy_score(y_test_list, y_pred_lin)

print(accuracy_rbf, accuracy_poly, accuracy_lin)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_pred=y_pred_rbf, y_true=y_test_list))
print(classification_report(y_pred=y_pred_poly,y_true= y_test_list))
print(classification_report(y_pred=y_pred_lin, y_true=y_test_list))
